<a href="https://colab.research.google.com/github/rautaditya2606/NY_TAXi_Fare/blob/main/nyc_taxi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install opendatasets --quiet
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import matplotlib
%matplotlib inline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
import opendatasets as od
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor, plot_tree, export_text
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, root_mean_squared_error
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
import random

sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10,6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

In [ ]:
tgdatabase_url = 'https://www.kaggle.com/c/new-york-city-taxi-fare-prediction'
od.download(database_url)

Skipping, found downloaded files in "./new-york-city-taxi-fare-prediction" (use force=True to force download)


In [ ]:
data_dir = '/content/new-york-city-taxi-fare-prediction/'

In [ ]:
!head {data_dir + '/train.csv'}

key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.84161,40.712278,1
2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.76127,-73.991242,40.750562,2
2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.98713,40.733143,-73.991567,40.758092,1
2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1
2011-01-06 09:50:45.0000002,12.1,2011-01-06 09:50:45 UTC,-74.000964,40.73163,-73.972892,40.758233,1
2012-11-20 20:35:00.0000001,7.5,2012-11-20 20:35:00 UTC,-73.980002,40.751662,-73.973802,40.764842,1
2012-01-04 17:22:00.00000081,16.5,2012-01-04 17:22:00 UTC,-73.9513,40.774138,-73.990095,40.751048,1
2012-12-03 13:10:00.000000125,9,2012-12-03 13:10:00 UTC,-74.006462,40.726713,-73.99

In [ ]:
selected_cols = 'fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count'.split(',')

In [ ]:
selected_cols
dtypes = {
 'fare_amount': 'float32',
 'pickup_longitude':'float32',
 'pickup_latitude':'float32',
 'dropoff_longitude':'float32',
 'dropoff_latitude':'float32',
 'passenger_count': 'uint8'
}

In [ ]:
def skip_rows(row_idx):
    if row_idx == 0:
        return False
    return random.random() > 0.01
df = pd.read_csv(
    data_dir + '/train.csv',
    usecols=selected_cols,
    dtype=dtypes,
    parse_dates=['pickup_datetime'],
    skiprows=skip_rows
)


In [ ]:
df

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,3.3,2012-07-18 00:16:00+00:00,-73.986656,40.756073,-73.989723,40.751701,5
1,17.0,2013-07-08 21:24:00+00:00,-74.009727,40.705166,-73.970894,40.749306,1
2,11.7,2012-06-24 17:33:00+00:00,-73.954498,40.769855,-73.984314,40.746006,1
3,10.0,2014-02-01 21:06:00+00:00,-73.979019,40.777668,-73.992172,40.721554,5
4,4.5,2013-06-20 12:36:08+00:00,-73.988449,40.770844,-73.985985,40.777706,1
...,...,...,...,...,...,...,...
553548,8.0,2013-01-14 15:11:43+00:00,-73.974213,40.747402,-73.999535,40.738495,1
553549,13.0,2013-05-27 09:45:40+00:00,-73.992775,40.750931,-73.956009,40.765732,1
553550,12.9,2011-09-07 00:04:00+00:00,-73.990746,40.749355,-73.965714,40.792149,5
553551,9.3,2009-06-25 21:28:00+00:00,-73.989853,40.735535,-73.974739,40.761799,2


In [ ]:
test_df = pd.read_csv(data_dir + '/test.csv', parse_dates=['pickup_datetime'])

In [ ]:
test_df

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24+00:00,-73.973320,40.763805,-73.981430,40.743835,1
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24+00:00,-73.986862,40.719383,-73.998886,40.739201,1
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44+00:00,-73.982524,40.751260,-73.979654,40.746139,1
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12+00:00,-73.981160,40.767807,-73.990448,40.751635,1
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12+00:00,-73.966046,40.789775,-73.988565,40.744427,1
...,...,...,...,...,...,...,...
9909,2015-05-10 12:37:51.0000002,2015-05-10 12:37:51+00:00,-73.968124,40.796997,-73.955643,40.780388,6
9910,2015-01-12 17:05:51.0000001,2015-01-12 17:05:51+00:00,-73.945511,40.803600,-73.960213,40.776371,6
9911,2015-04-19 20:44:15.0000001,2015-04-19 20:44:15+00:00,-73.991600,40.726608,-73.789742,40.647011,6
9912,2015-01-31 01:05:19.0000005,2015-01-31 01:05:19+00:00,-73.985573,40.735432,-73.939178,40.801731,6


In [ ]:
df.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,553553.000000,553553.000000,553553.000000,553551.000000,553551.000000,553553.000000
mean,11.332041,-72.521149,39.932858,-72.503822,39.936440,1.687439
std,9.753575,13.211610,9.046136,11.554820,8.481799,1.338966
min,-52.000000,-3050.561035,-2554.123291,-1343.457886,-1172.810059,0.000000
25%,6.000000,-73.992088,40.734940,-73.991394,40.734013,1.000000
50%,8.500000,-73.981834,40.752602,-73.980148,40.753174,1.000000
75%,12.500000,-73.967110,40.767136,-73.963692,40.768112,2.000000
max,280.000000,2417.441406,3347.258789,2497.105713,2604.858154,208.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 553553 entries, 0 to 553552
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype              
---  ------             --------------   -----              
 0   fare_amount        553553 non-null  float32            
 1   pickup_datetime    553553 non-null  datetime64[ns, UTC]
 2   pickup_longitude   553553 non-null  float32            
 3   pickup_latitude    553553 non-null  float32            
 4   dropoff_longitude  553551 non-null  float32            
 5   dropoff_latitude   553551 non-null  float32            
 6   passenger_count    553553 non-null  uint8              
dtypes: datetime64[ns, UTC](1), float32(5), uint8(1)
memory usage: 15.3 MB


In [ ]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
train_df.shape, val_df.shape

((442842, 7), (110711, 7))

In [ ]:
train_df = train_df.dropna()
val_df = val_df.dropna()

In [ ]:
list(train_df.columns)

['fare_amount',
 'pickup_datetime',
 'pickup_longitude',
 'pickup_latitude',
 'dropoff_longitude',
 'dropoff_latitude',
 'passenger_count']

In [ ]:
input_cols = [
 'pickup_longitude',
 'pickup_latitude',
 'dropoff_longitude',
 'dropoff_latitude',
 'passenger_count']
target_cols = 'fare_amount'

In [ ]:
train_inputs = train_df[input_cols]
train_targets = train_df[target_cols]
val_inputs = val_df[input_cols]
val_targets = val_df[target_cols]

KeyboardInterrupt: 

In [ ]:
def add_dateparts(df, col):
  df[col+'_year'] = df[col].dt.year
  df[col+'_month'] = df[col].dt.month
  df[col+'_day'] = df[col].dt.day
  df[col+'_weekday'] = df[col].dt.weekday
  df[col+'_hour'] = df[col].dt.hour

In [ ]:
add_dateparts(train_df, 'pickup_datetime')
add_dateparts(val_df, 'pickup_datetime')
add_dateparts(test_df, 'pickup_datetime')

In [ ]:
train_df

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_datetime_year,pickup_datetime_month,pickup_datetime_day,pickup_datetime_weekday,pickup_datetime_hour
400515,6.500000,2014-08-28 12:20:00+00:00,-73.994339,40.761337,-74.003181,40.750507,2,2014,8,28,3,12
396606,15.900000,2009-03-05 18:58:42+00:00,-73.993500,40.724564,-73.955742,40.763996,1,2009,3,5,3,18
470070,37.660000,2014-07-07 21:24:00+00:00,-73.863068,40.769096,-73.988670,40.744560,1,2014,7,7,0,21
340305,8.000000,2015-01-16 20:03:32+00:00,-74.001823,40.724655,-73.991776,40.744164,5,2015,1,16,4,20
138225,9.500000,2012-10-12 20:04:00+00:00,-73.980103,40.752609,-73.994148,40.746021,1,2012,10,12,4,20
...,...,...,...,...,...,...,...,...,...,...,...,...
110268,22.500000,2013-07-20 05:02:41+00:00,-73.996178,40.716377,-73.969879,40.794632,1,2013,7,20,5,5
259178,4.100000,2010-08-08 22:39:28+00:00,-73.991608,40.735123,-73.980316,40.730324,1,2010,8,8,6,22
365838,6.900000,2010-04-25 02:49:05+00:00,-73.955246,40.785870,-73.981010,40.784534,1,2010,4,25,6,2
131932,21.299999,2012-05-14 00:08:00+00:00,-73.981544,40.758904,-73.955284,40.689487,1,2012,5,14,0,0


In [ ]:
val_df

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_datetime_year,pickup_datetime_month,pickup_datetime_day,pickup_datetime_weekday,pickup_datetime_hour
534301,14.500000,2011-10-01 12:45:00+00:00,-74.006027,40.735786,-73.975327,40.780231,2,2011,10,1,5,12
291434,27.299999,2009-01-09 20:03:00+00:00,-73.874573,40.774155,-74.008499,40.778290,1,2009,1,9,4,20
185153,8.500000,2009-08-24 13:43:15+00:00,-73.967102,40.760929,-73.977715,40.783043,1,2009,8,24,0,13
13432,10.500000,2009-04-06 07:08:59+00:00,0.000000,0.000000,0.000000,0.000000,1,2009,4,6,0,7
252011,13.500000,2013-05-03 00:31:00+00:00,-73.970863,40.754814,-74.002899,40.725761,1,2013,5,3,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...
424553,7.000000,2013-01-10 08:48:09+00:00,-73.988823,40.774197,-73.977966,40.782921,1,2013,1,10,3,8
247254,4.900000,2009-10-20 09:01:40+00:00,-73.971970,40.765446,-73.973007,40.758934,1,2009,10,20,1,9
58124,49.799999,2011-12-21 19:13:32+00:00,-73.966782,40.753418,-73.789619,40.642906,1,2011,12,21,2,19
241454,60.250000,2013-11-12 22:41:00+00:00,-73.973068,40.760651,-74.038872,40.742863,1,2013,11,12,1,22


In [ ]:
test_df

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_datetime_year,pickup_datetime_month,pickup_datetime_day,pickup_datetime_weekday,pickup_datetime_hour
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24+00:00,-73.973320,40.763805,-73.981430,40.743835,1,2015,1,27,1,13
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24+00:00,-73.986862,40.719383,-73.998886,40.739201,1,2015,1,27,1,13
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44+00:00,-73.982524,40.751260,-73.979654,40.746139,1,2011,10,8,5,11
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12+00:00,-73.981160,40.767807,-73.990448,40.751635,1,2012,12,1,5,21
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12+00:00,-73.966046,40.789775,-73.988565,40.744427,1,2012,12,1,5,21
...,...,...,...,...,...,...,...,...,...,...,...,...
9909,2015-05-10 12:37:51.0000002,2015-05-10 12:37:51+00:00,-73.968124,40.796997,-73.955643,40.780388,6,2015,5,10,6,12
9910,2015-01-12 17:05:51.0000001,2015-01-12 17:05:51+00:00,-73.945511,40.803600,-73.960213,40.776371,6,2015,1,12,0,17
9911,2015-04-19 20:44:15.0000001,2015-04-19 20:44:15+00:00,-73.991600,40.726608,-73.789742,40.647011,6,2015,4,19,6,20
9912,2015-01-31 01:05:19.0000005,2015-01-31 01:05:19+00:00,-73.985573,40.735432,-73.939178,40.801731,6,2015,1,31,5,1


In [ ]:
def haversine_vectorized(lat1, lon1, lat2, lon2):
    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6371 * c
    return km

def add_trip_dist(df):
    df['trip_distance'] = haversine_vectorized(
        df['pickup_latitude'],
        df['pickup_longitude'],
        df['dropoff_latitude'],
        df['dropoff_longitude']
    )


In [ ]:
add_trip_dist(train_df)
add_trip_dist(val_df)
add_trip_dist(test_df)

In [ ]:
 train_df

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_datetime_year,pickup_datetime_month,pickup_datetime_day,pickup_datetime_weekday,pickup_datetime_hour,trip_distance
400515,6.500000,2014-08-28 12:20:00+00:00,-73.994339,40.761337,-74.003181,40.750507,2,2014,8,28,3,12,1.416000
396606,15.900000,2009-03-05 18:58:42+00:00,-73.993500,40.724564,-73.955742,40.763996,1,2009,3,5,3,18,5.417093
470070,37.660000,2014-07-07 21:24:00+00:00,-73.863068,40.769096,-73.988670,40.744560,1,2014,7,7,0,21,10.925797
340305,8.000000,2015-01-16 20:03:32+00:00,-74.001823,40.724655,-73.991776,40.744164,5,2015,1,16,4,20,2.328424
138225,9.500000,2012-10-12 20:04:00+00:00,-73.980103,40.752609,-73.994148,40.746021,1,2012,10,12,4,20,1.392073
...,...,...,...,...,...,...,...,...,...,...,...,...,...
110268,22.500000,2013-07-20 05:02:41+00:00,-73.996178,40.716377,-73.969879,40.794632,1,2013,7,20,5,5,8.979382
259178,4.100000,2010-08-08 22:39:28+00:00,-73.991608,40.735123,-73.980316,40.730324,1,2010,8,8,6,22,1.091216
365838,6.900000,2010-04-25 02:49:05+00:00,-73.955246,40.785870,-73.981010,40.784534,1,2010,4,25,6,2,2.174174
131932,21.299999,2012-05-14 00:08:00+00:00,-73.981544,40.758904,-73.955284,40.689487,1,2012,5,14,0,0,8.029993
